# Training Notebook
Notebook for training the acoustic model of my speech 2 text AI.
### To-Do
* Build data loaders
* Write code to convert wav to log mel spectrograms > separate module
* Add SpecAugment to improve training
* Build acoustic model > separate module
* Training loop
* Validation loop
* Training visualization

In [1]:
import torch
import torchaudio
import torch.nn as nn
import pandas as pd
import numpy as np
import utilities

In [2]:
textprocess = utilities.TextProcess()

In [3]:
textprocess.text_to_int_seq('hello')

[9, 6, 13, 13, 16]

In [4]:
textprocess.int_to_text_seq([9, 6, 13, 13, 16])

'hello'